In [1]:
import numpy as np
import pandas as pd

data_imported = pd.read_csv("data/kickstart_join_result.csv", sep="\t", error_bad_lines=False, encoding="utf-8" )
data_imported.head(10)

b'Skipping line 25531: expected 18 fields, saw 19\n'
b'Skipping line 231615: expected 18 fields, saw 19\n'


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real,url,blurb,slug,creator_name
0,Rocky Mountain Trails and Campsite Poster,Graphic Design,Design,USD,2015-07-13 00:00:00,6000.0,2015-06-12 05:48:00,100.00,failed,5,US,100.00,100.00,6000.00,https://www.kickstarter.com/projects/euclidmap...,The trails and campsites of Rocky Mountain Nat...,rocky-mountain-trails-and-campsite-poster,David H
1,Millennial Hunter Podcast,Radio & Podcasts,Publishing,USD,2017-08-16 00:00:00,1500.0,2017-07-17 23:12:00,158.00,failed,4,US,78.00,158.00,1500.00,https://www.kickstarter.com/projects/131721437...,Helping the world see that millennials and hun...,millennial-hunter-podcast,Colby Skinner
2,Brethren Born#3:Deceptions,Comic Books,Comics,GBP,2017-04-02 00:00:00,1500.0,2017-03-03 13:06:00,2952.00,successful,139,GB,2155.64,3667.04,1863.33,https://www.kickstarter.com/projects/212408651...,BB#3 is full of intrigue adventure & action as...,brethren-born3-deceptions,Jon Laight
3,The Land without Color,Children's Books,Publishing,USD,2015-09-17 00:00:00,8000.0,2015-08-18 13:19:00,8360.00,successful,77,US,8360.00,8360.00,8000.00,https://www.kickstarter.com/projects/ellefsonb...,Adventure novel for children of all ages that ...,the-land-without-color,Benjamin Ellefson
4,Boxing Game: The Pugilist Path,Video Games,Games,USD,2015-11-11 00:00:00,82000.0,2015-10-12 19:32:00,189.00,failed,3,US,189.00,189.00,82000.00,https://www.kickstarter.com/projects/708033966...,A retro style boxing management tycoon game wh...,boxing-game-the-pugilist-path,industrialZenLLC
5,CAMBALACHE Son Jarocho in LA,World Music,Music,USD,2013-04-24 00:00:00,12000.0,2013-03-07 00:43:00,13588.33,successful,241,US,13588.33,13588.33,12000.00,https://www.kickstarter.com/projects/131820506...,Cambalache Son Jarocho led by Cesar Castro is ...,cambalache-son-jarocho-in-la,cesar castro cambalache
6,Book Project: Enough is Enough Already,Fiction,Publishing,USD,2013-07-23 00:00:00,2000.0,2013-07-18 07:57:00,13.00,failed,4,US,13.00,13.00,2000.00,https://www.kickstarter.com/projects/thecfproj...,Story of faith. Loreina encounters the Lord on...,book-project-enough-is-enough-already,Carline Francois
7,Sartor London tailors & luxury garment makers,Apparel,Fashion,GBP,2017-06-11 00:00:00,2000.0,2017-05-12 19:04:00,0.00,failed,0,GB,0.00,0.00,2553.82,https://www.kickstarter.com/projects/127272689...,Sartor London's vision is to produce beautiful...,sartor-london-tailors-and-luxury-garment-makers,james armstrong
8,No Place to Hide: The Video Games of Hideo Kojima,Nonfiction,Publishing,USD,2015-04-10 00:00:00,6000.0,2015-03-10 01:58:00,28.00,failed,3,US,28.00,28.00,6000.00,https://www.kickstarter.com/projects/509596074...,'A technical and thematic analysis of the work...,no-place-to-hide-the-video-games-of-hideo-kojima,Jake Weston
9,Green Thumb Cook,Cookbooks,Food,USD,2016-02-23 00:00:00,15000.0,2016-01-24 19:55:00,0.00,failed,0,US,0.00,0.00,15000.00,https://www.kickstarter.com/projects/871669244...,Recipies and Memories from Around the World. A...,green-thumb-cook,Denny Klatt


In [2]:
def remove_those_nan(string):
    if isinstance(string, float):
        if np.isnan(string):
            print("That's a nan")
        return False
    else:
        return True

#print(data_imported[data_imported["blurb"].apply(is_not_ascii)])
data_imported_nonan = data_imported.copy()
data_imported_nonan.loc[:, "blurb"] = data_imported_nonan.blurb.dropna(how="any")
data_imported_nonan = data_imported_nonan[data_imported_nonan.blurb.apply(remove_those_nan)]
print(data_imported_nonan.shape[0])
print(data_imported.shape[0])

That's a nan
That's a nan
That's a nan
248408
248411


## Goals for this notebook
<ul>
    <li>Use a naive bayes algorithm to predict the successfulness based on the blurb</li>
    <ul>
        <li>If this works I will attempt to extract the most sensitive words for this</li>
        <li>I would also give it a try to see if I can run an algorithm on the categories</li>
    </ul>
    <li>I would also like to give a try on what categories each belongs to</li>
</ul>

In [3]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report

# Bring in the stemmer and grab the stopwords
import nltk
from nltk.stem.snowball import EnglishStemmer
nltk.download("stopwords")

import re

# Split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Encode y
Encoder = LabelEncoder()
y_encode = Encoder.fit_transform(data_imported_nonan.state)
X_train, X_test, y_train, y_test = train_test_split(data_imported_nonan.blurb, 
                                                    y_encode, 
                                                    test_size=0.2, 
                                                    random_state=42)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sora4\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
stemmer = EnglishStemmer(ignore_stopwords=True)
def stem_tokenizer(text):
    text_to_output = text.lower()
    
    # Removes everything that is not a word or a number
    text_to_output = re.sub("[^\w\ ]", "" ,text_to_output)
    return [stemmer.stem(word) for word in text_to_output.split(" ")]

vect = HashingVectorizer(decode_error='ignore', n_features=2**25, tokenizer=stem_tokenizer, alternate_sign=False)

In [5]:
# Obtain the vectorized data
X_train_vect = vect.transform(X_train)

In [23]:
bayes_clf = MultinomialNB()
param_range = [0.001, 0.003, 0.01, 0.03, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 1, 2, 3, 4, 10, 100, 1.5, 3.5, 2.5]
param_grid = [{'alpha': param_range}]

# Searches for best parameters
print("In we go")
gs = GridSearchCV(estimator=bayes_clf,
                  param_grid=param_grid,
                  scoring='roc_auc',
                  cv=10,
                  n_jobs=-1)
gs.fit(X_train_vect, y_train)
print(gs.best_score_)
print(gs.best_params_)

In we go
0.7241602276139723
{'alpha': 0.4}


In [24]:
X_test_vect = vect.transform(X_test)
y_pred = gs.best_estimator_.predict(X_test_vect)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.61      0.85      0.71     25677
          1       0.72      0.42      0.53     24005

avg / total       0.66      0.64      0.62     49682



# So Naive Bayes Failed....
This isn't horrible, the fact that it failed can be due to two factors:
1. The premise that $P(A \cap B ) = P(A)P(B)$ is untrue, this is most likely
2. The hasing vectorizer, (which upon collision makes the numbers negative) I had to stop, this leads to the hashing vectorizer not giving the right count for each word.
I can try to fix this by using TF-IDF, I doubt it will however.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
vect = TfidfVectorizer(tokenizer=stem_tokenizer)
X_train_vect = vect.fit_transform(X_train)

In [11]:
param_range = [0.001, 0.003, 0.01, 0.03, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 1, 2, 3, 4, 10, 100, 1.5, 3.5, 2.5]
param_grid = [{'alpha': param_range}]
# Searches for best parameters
print("In we go")
bayes_clf = MultinomialNB()
gs = GridSearchCV(estimator=bayes_clf,
                  param_grid=param_grid,
                  scoring='roc_auc',
                  cv=10,
                  n_jobs=-1)
gs.fit(X_train_vect, y_train)
print(gs.best_score_)
print(gs.best_params_)
X_test_vect = vect.transform(X_test)
y_pred = gs.best_estimator_.predict(X_test_vect)
print(classification_report(y_test, y_pred))

In we go
0.7271993624382251
{'alpha': 2}
             precision    recall  f1-score   support

          0       0.64      0.77      0.70     25677
          1       0.68      0.54      0.61     24005

avg / total       0.66      0.66      0.65     49682



# That didn't go great. On to better algorithms!
I will give logistic regression a chance, it is usually a pretty good algorithm in general. I could also try SVC. For selecting algorithms, it is usually better to try with a k-folds approach.

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [14]:
forest = RandomForestClassifier(n_jobs=-1)
svc = SVC()
log_reg = LogisticRegression(n_jobs=-1)

In [15]:
# I will stick with the tf-idf
def scoring(estimator, param_grid, n_jobs=1):
    gs = GridSearchCV(estimator=estimator, 
                      param_grid=param_grid,
                      n_jobs=n_jobs, 
                      cv=2)
    score = cross_val_score(gs, X_train_vect, y_train, scoring="accuracy", cv=3, n_jobs=n_jobs)
    print("CV {} accuracy: {:.3f} +/- {:.2f}".format(type(estimator).__name__ ,np.mean(score), np.std(score)))
    
    return gs

In [16]:
forest_pg = {"n_estimators":[10, 30, 100, 300, 1000], "max_depth": [None, 10, 30, 100, 300]}
svc_pg = {"C": [0.1, 0.3, 1.0, 3.0, 10.0, 30.0, 100.0], "gamma": [0.001, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 100.0, 1000.0], "kernel":["rbf"]}
log_pg = {"C": [0.1, 0.3, 1.0, 3.0, 10.0, 100.0], "max_iter": [100, 300]}

In [17]:
gs_forest = scoring(forest, forest_pg, 1)

KeyboardInterrupt: 

In [ ]:
gs_svc = scoring(svc, svc_pg, -1)

In [ ]:
gs_log = scoring(log_reg, log_pg, 1)